In [ ]:
# python3 -m pip install --user --upgrade matplotlib

In [ ]:
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Prep-Work

### Import Data

In [ ]:
# In this notebook we only process the main signal and the generic events,
# for illustration purposes.
# You can add other backgrounds after if you wish.
samples = ["Signal", "ccbar"]
# samples = ["Signal", "ccbar","charged","ddbar","mixed","ssbar","taupair","uubar"]

In [ ]:
DataFrames = {}  # define empty dictionary to hold dataframes
Date = "0213"
Attempt = "2"

# Signal:
DataFrames[samples[0]] =  uproot.concatenate("/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root:Dstree",library='pd')
# Background
for s in samples[1:]: # loop over samples
    DataFrames[s] =  uproot.concatenate("/group/belle2/users2022/amubarak/02-Grid/Completed_TopoAna/Ds2D0e-Generic_Ds_" + Date +"25_"+ Attempt +"_"+ s +".root:Dstree",library='pd')

In [ ]:
# Right now it is set to look at signal.
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)

The line below is to look at the available variables.

In [ ]:
DataFrames["All"].columns.tolist()

### Setup
The code below will be used to apply cuts to the data.  
The range of the plots.

In [ ]:
# Electron ID
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['e_electronID']>=0.95]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['e_electronID']>=0.95]
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_em_electronID']>=0.95]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_gammaveto_em_electronID']>=0.95]

# Photon Conversion
#-------------------
DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=0.1]
DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_gammaveto_M_Correction']>=0.1]

# Peaking Background Removal
#----------------------------
# DataFrames["All"] = DataFrames["All"][(DataFrames["All"]['Ds_diff_D0pi']>=0.15)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_diff_D0pi']>=0.15)]

# # Vertex Fitting
# #----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb']>=0.01]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_chiProb']>=0.01]

# Dalitz Removal
#----------------------------
# DataFrames["All"] = DataFrames["All"][(DataFrames["All"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["All"]['Ds_pi0veto_M_Correction']>=0.16)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["Signal"]['Ds_pi0veto_M_Correction']>=0.16)]

# Vertex Fit
#----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb_Ds_rank']==1]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_chiProb_Ds_rank']==1]

# D0 Invariant Mass
#-----------------------
DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_D0_sideband']==1)]
DataFrames["All"] = DataFrames["All"][(DataFrames["All"]['Ds_D0_sideband']==1)]

# KaonID
#-----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['K_kaonID']>=0.5]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['K_kaonID']>=0.5]

In [ ]:
DataFrames["All"].isna().sum()

# Electron

### Prep-Work

In [ ]:
print(DataFrames["All"]['e_genMotherPDG'].value_counts(dropna=False))

In [ ]:
# Count all values in e_mcErrors column
print(DataFrames["All"]['e_mcErrors'].value_counts(dropna=False))

### Breakdown

In [ ]:
Sample = "All"
Norm = False
Show = 10

Op = 0.58
dM = 0.1
Hits = 0

In [ ]:
# Count all values in column for e_mcPDG for specfic code
print(DataFrames[Sample][(abs(DataFrames[Sample]['e_mcSecPhysProc'])==16)]['e_mcPDG'].value_counts(normalize=Norm,dropna=False))

In [ ]:
# Count all values in column for e_mcPDG for specfic code
print(DataFrames[Sample]['e_mcSecPhysProc'].value_counts(normalize=Norm,dropna=False))
print(DataFrames[Sample][(abs(DataFrames[Sample]['e_mcPDG'])==11)]['e_mcSecPhysProc'].value_counts(normalize=Norm,dropna=False))

In [ ]:
# Count all values in column for e_genMotherPDG for specfic code
print(abs(DataFrames[Sample][(abs(DataFrames[Sample]['e_mcPDG'])==11)]['e_genMotherPDG']).value_counts(normalize=Norm,dropna=False))

In [ ]:
print(abs(DataFrames[Sample][(abs(DataFrames[Sample]['e_mcPDG'])==11) & (DataFrames[Sample]['e_genMotherPDG']==22)]['e_genMotherPDG_1']).value_counts(normalize=Norm,dropna=False))

# Non-Tracks

## mcError
- mcError=512: 
    - The largest category is for when a reconstructed particle cannot be matched to an MC particle. 
- mcError=496: (c_MissGamma = 16 + c_MissMassiveParticle = 32 + c_MissKlong = 64 + c_MisID = 128 + c_AddedWrongParticle = 256)
- mcError=432: (c_MissGamma = 16 + c_MissMassiveParticle = 32 + c_MisID = 128 + c_AddedWrongParticle = 256)
- mcError=384 (c_MisID = 128 + c_AddedWrongParticle = 256): 
    - When a pion faking an electron is combined with $D^{0}$
- mcError=304 (c_MissGamma = 16 + c_MissMassiveParticle = 32 + c_AddedWrongParticle = 256)
- mcError=272 (c_MissGamma = 16 + c_AddedWrongParticle = 256)

In [ ]:
dM = 0.1
print("Number of 23:",len(DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==23) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM)]))
print("Number of 423:",len(DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==423) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM)]))
print("Number of 413:",len(DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==413) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM)]))
print("Number of NaN:",len( DataFrames["ccbar"][(DataFrames["ccbar"]["Ds_mcPDG"].isna()) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM)]))
print("Number of Other:",len(DataFrames["ccbar"][(abs(DataFrames["ccbar"]['Ds_Other'])==1) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM)]))

In [ ]:
print("Failed+Comb: ", len(DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==23) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM)])+len(DataFrames["ccbar"][(DataFrames["ccbar"]["Ds_mcPDG"].isna()) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM)]))

In [ ]:
print(DataFrames["ccbar"]['Ds_mcErrors'].value_counts(normalize=Norm,dropna=False))

In [ ]:
print(abs(DataFrames["All"]['Ds_mcPDG']).value_counts(normalize=True,dropna=False).head(100))

In [ ]:
Op = 0.58
dM = 0.1
Hits = -1
Norm = True

In [ ]:
print(abs(DataFrames["All"][(abs(DataFrames["All"]['Ds_Other'])==1) & (abs(DataFrames["All"]['Ds_mcPDG'])!=300553) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op)]['D0_mcPDG']).value_counts(normalize=Norm,dropna=False).head(Show))
print(abs(DataFrames["All"][(abs(DataFrames["All"]['Ds_Other'])==1) & (abs(DataFrames["All"]['Ds_mcPDG'])!=300553) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op)]['e_mcPDG']).value_counts(normalize=Norm,dropna=False).head(Show))
print(abs(DataFrames["All"][(abs(DataFrames["All"]['Ds_Other'])==1) & (abs(DataFrames["All"]['Ds_mcPDG'])!=300553) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op)]['K_mcPDG']).value_counts(normalize=Norm,dropna=False).head(Show))
print(abs(DataFrames["All"][(abs(DataFrames["All"]['Ds_Other'])==1) & (abs(DataFrames["All"]['Ds_mcPDG'])!=300553) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op)]['pi_mcPDG']).value_counts(normalize=Norm,dropna=False).head(Show))

In [ ]:
print(abs(DataFrames["All"][(abs(DataFrames["All"]['Ds_Other'])==1) & (abs(DataFrames["All"]['Ds_mcPDG'])!=300553) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op)]['D0_genMotherPDG']).value_counts(normalize=Norm,dropna=False).head(Show))
print(abs(DataFrames["All"][(abs(DataFrames["All"]['Ds_Other'])==1) & (abs(DataFrames["All"]['Ds_mcPDG'])!=300553) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op)]['e_genMotherPDG']).value_counts(normalize=Norm,dropna=False).head(Show))
print(abs(DataFrames["All"][(abs(DataFrames["All"]['Ds_Other'])==1) & (abs(DataFrames["All"]['Ds_mcPDG'])!=300553) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op)]['K_genMotherPDG']).value_counts(normalize=Norm,dropna=False).head(Show))
print(abs(DataFrames["All"][(abs(DataFrames["All"]['Ds_Other'])==1) & (abs(DataFrames["All"]['Ds_mcPDG'])!=300553) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op)]['pi_genMotherPDG']).value_counts(normalize=Norm,dropna=False).head(Show))

In [ ]:
print(abs(DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])==23) & (abs(DataFrames["All"]['D0_genMotherPDG'])==423) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op)]['e_genMotherPDG']).value_counts(normalize=Norm,dropna=False).head(Show))
print(abs(DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])==23) & (abs(DataFrames["All"]['D0_genMotherPDG'])==413) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op)]['e_genMotherPDG']).value_counts(normalize=Norm,dropna=False).head(Show))
print(abs(DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])==23) & (abs(DataFrames["All"]['D0_genMotherPDG'])==23) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op)]['e_genMotherPDG']).value_counts(normalize=Norm,dropna=False).head(Show))

# Combinatoric (ID Check) 

In [ ]:
ID = ['e_genMotherPDG','e_genMotherID','e_genMotherPDG_1','e_genMotherID_1','e_genMotherPDG_2','e_genMotherID_2','D0_genMotherPDG','D0_genMotherID','Ds_mcPDG','Ds_genParticleID']

In [ ]:
DataFrames["All"][(abs(DataFrames["All"]['Ds_Other'])==1) & (abs(DataFrames["All"]['Ds_mcPDG'])!=300553)][ID].head(30)

# Electron Purity

In [ ]:
# x = np.linspace(0.8, 1, 10, endpoint = True) 
# Electron_frac = np.zeros(len(x))
# Other_frac = np.zeros(len(x))

# for i in range(len(x)):
#     Electron = df_ccbar[(df_ccbar['e_electronID'] >= x[i]) & (abs(df_ccbar['e_mcPDG'])==11)].shape[0]
#     Other = df_ccbar[(df_ccbar['e_electronID'] >= x[i]) & (abs(df_ccbar['e_mcPDG'])!=11)].shape[0]

#     Electron_frac[i] = Electron/(Electron + Other)
#     Other_frac[i] = Other/(Electron + Other)

# # Create a line plot
# plt.plot(x, Electron_frac)
# plt.plot(x, Other_frac)

# plt.xlabel('electron_ID')
# plt.ylabel('Fraction')
# plt.title('Purity')
# # Show the plot
# plt.show()